In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from tkinter import filedialog,Text
from PIL import Image, ImageOps
import tkinter as tk
import io

In [ ]:
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten
import tensorflow_hub as hub
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dot, Lambda, Input
from tensorflow.keras.models import Sequential,Model
import keras
from tensorflow.keras import datasets, layers, models

In [ ]:
labels = pd.read_csv('../input/dog-breed-identification/labels.csv')
labels.head()

In [ ]:
train_file_location = '../input/dog-breed-identification/train/' 
train_data = labels.assign(img_path = lambda x : train_file_location + x['id'] + '.jpg')
train_data.head()

In [ ]:
train_data.iloc[3,2][2:]

In [ ]:
def image_preocessing(path):
    image = Image.open(path)
    size = (128, 128)
    image = ImageOps.fit(image, size, Image.ANTIALIAS)
    #image= image.transpose(Image.FLIP_LEFT_RIGHT)
    image_array = np.asarray(image)
    a,b,c = (np.shape(image_array))
    if c==4:
        image_array = image_array[:,:,:3]
    image_array = (image_array.astype(np.float32) / 255.0) 
    
    return image_array

In [ ]:
dir='../input/dog-breed-identification/train'
prevfolder='none'
label=0
dogs = []
label_names=[]

for i in range(0,len(train_data)):
    label=train_data.iloc[i,1]
    filename=train_data.iloc[i,2]
    #print(filename)
    image = image_preocessing(filename)
    dogs.append(image)
    label_names.append(label)

print(labels)
print(len(labels),"images")

In [ ]:
print(np.shape(label_names))
print(np.shape(dogs))

In [ ]:
Y = pd.get_dummies(label_names)

In [ ]:
X_train=np.array(dogs)
dogs=5
Y_train=Y.to_numpy()

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test  = train_test_split(X_train,Y_train,test_size = 0.25)

In [ ]:
print(X_train.shape,Y_train.shape)

In [ ]:
Y_train[4]

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
train_datagen.fit(X_train)

train_generator = train_datagen.flow(X_train,Y_train,batch_size = 32)

In [ ]:
model = keras.models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128,128, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((4, 4)))
model.add(layers.Flatten())
model.add(layers.Dense(len(np.unique(label_names)), activation="softmax"))

  
    
    
model.summary()

In [ ]:
model.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

In [ ]:
model.fit(train_generator,
         validation_data=(X_test, Y_test),
         epochs=1,
         verbose=2)

In [ ]:
submission = pd.read_csv('../input/dog-breed-identification/sample_submission.csv')

In [ ]:
submission.head()

In [ ]:
test_datagen = ImageDataGenerator ( 
    rescale=1./255
)

test_set = test_datagen.flow_from_directory(
    '/kaggle/input/dog-breed-identification',
    target_size = (128,128),
    batch_size = 128,
    classes=['test']
)

In [ ]:
test_datagen = ImageDataGenerator ( 
    rescale=1./255
)

test_set = test_datagen.flow_from_directory(
    '/kaggle/input/dog-breed-identification',
    target_size = (128,128),
    batch_size = 128,
    classes=['test']
)

In [ ]:
y_pred = model.predict(test_set)
y_pred.shape

In [ ]:
import re
file_list = test_set.filenames
id_list = []
for name in file_list:
    m = re.sub('test/', '', name)
    m = re.sub('.jpg', '', m)
    id_list.append(m)

In [ ]:
submission['id'] = id_list
submission.iloc[:,1:] =y_pred
submission.head()

In [ ]:
final_df = submission.set_index('id')
final_df.to_csv('Submission.csv')